In [7]:
import findspark
findspark.init()
findspark.find()
import datetime
import decimal
import os
import pyspark

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType
from pyspark.sql.types import StructType,StructField, StringType, IntegerType ,LongType
from typing import List

In [16]:
spark = SparkSession.builder.appName('Optimize I').getOrCreate()

base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

answers_input_path = os.path.join(project_path, 'C:/Users/KEVAL/Desktop/Optimization/data/answers')    

questions_input_path = os.path.join(project_path, 'C:/Users/KEVAL/Desktop/Optimization/data/questions' )

answersDF = spark.read.option('C:/Users/KEVAL/Desktop/Optimization/data/answers/', answers_input_path).load()

questionsDF = spark.read.option('C:/Users/KEVAL/Desktop/Optimization/data/questions/', questions_input_path).load()

'''
Answers aggregation
Here we : get number of answers per question per month
'''

answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

# questionsDF has 4 partitions and answers_month has 200. Uisng coalesce, we reduce the number of partitions which reduces the number of shuffles for the join transformation
answers_month=answers_month.coalesce(4)

# Removed column 'creation_date'
resultDF = questionsDF.join(answers_month, 'question_id').select('question_id', 'title', 'month', 'cnt')

resultDF.orderBy('question_id', 'month').show(truncate=False)

resultDF.explain()

22/05/02 22:46:51 WARN DataSource: All paths were ignored:
  


AnalysisException: Unable to infer schema for Parquet. It must be specified manually.